## Interfaces con clases en C++

El ejemplo original está [acá](https://stackoverflow.com/questions/602580/how-can-i-use-c-class-in-python) que sigue [este ejemplo](https://www.auctoris.co.uk/2017/04/29/calling-c-classes-from-python-with-ctypes/):

El código en C++ que tenemos es:

```c++
class Test{
     private:
        int n;
     public:
        Test(int k){
            n=k;
        }
        void setInt(int k){
            n = k;
        }
        int getInt(){
            return n;
        }
};
```

In [3]:
cd ../scripts/interfacing_Cpp

/Users/flavioc/Library/Mobile Documents/com~apple~CloudDocs/Documents/cursos/Python/intro-python-IB/scripts/interfacing_Cpp


La implementación de Python que estamos usando está escrita en C, de modo tal que tenemos que exportar las funciones de la clase `Test` en C++ en el código fuente de la siguiente manera:

```c++
extern "C" 
{
    // include below each method you want to make visible outside
    Test* init(int k) {return new Test(k);}
    void setInt(Test *self, int k) {self->setInt(k);}
    int getInt(Test *self) {return self->getInt();}
    
    // Add the declaration '__declspec(dllexport)' before each function in Windows
}
```

La declaración `extern "C"` indican al compilador de C++ que genere código compatible con C de todas las funciones incluídas en el bloque. 

### CTypes

Vamos ahora a usar directamente `Ctypes`. Como antes, el primer paso es compilar nuestro código y generar una biblioteca:
```bash
$ g++ -fpic -shared test.cpp -o libtest.so
```

Si uno trabaja en Windows, generará una dll

```cmd
cl.exe -c test.cpp
link.exe /DLL /OUT:test.dll
```


In [ ]:
# !gcc -fpic -Wall -shared rotacion.c -o librotacion.so
!g++ -fpic -shared test.cpp -o libtest.so



In [ ]:
!ls

En segundo lugar, importamos el módulo `ctypes`

In [4]:
import ctypes

Este módulo nos provee de la función `CDLL` para importar la biblioteca

In [5]:
lib = ctypes.CDLL('./libtest.so')

Ahora vamos a crear una clase en Python equivalente a la que teníamos en C++. Al igual que en el caso de C, tenemos que establecer los tipos de datos de entrada (via el método `argtypes`) y salida (vía el método `restype`) para _cada función de la clase_. 

In [6]:

class Test():
    def __init__(self, val: int):
        # Declare input and output types for each method you intend to use
        lib.init.argtypes = [ctypes.c_int]
        lib.init.restype = ctypes.c_void_p

        lib.setInt.argtypes = [ctypes.c_void_p, ctypes.c_int]
        lib.setInt.restype = ctypes.c_void_p

        lib.getInt.argtypes = [ctypes.c_void_p]
        lib.getInt.restype = ctypes.c_int
        
        # use the C++ constructor to build the instance 
        self.q = lib.init(val)

    def setInt(self, n):
        lib.setInt(self.q, n)
    
    def getInt(self):
        return lib.getInt(self.q)


In [7]:
T1 = Test(12)
print(T1.getInt())
T1.setInt(32)
print(T1.getInt())

12
32


In [9]:
type(T1.q)

int